# Uso de la API

In [ ]:
from BKAudienceConnector import BK_API as bk
import numpy as np
import pandas as pd
import json

La base del codigo son 2 funciones.

* __InputBuilder__
* __doRequest__

las cuales construyen la url para la API y hacen la consulta respectivamente.

# Descripcion del codigo

In [ ]:
bk.InputBuilder?

In [ ]:
bk.doRequest?

# Codigo Fuente del Codigo

In [ ]:
bk.InputBuilder??

In [ ]:
bk.doRequest??

# Llaves de Autenticación

In [ ]:
bkuid =  #Web Service User Key
bksecretkey = #Web Service Private Key
partner_id =  # Partner Id

# Revisando que tenemos conexión
Para esto usaremos el método de PING proporcionado por la URL


``` http://services.bluekai.com/Services/WS/Ping?pid= ```

Para esto hacemos uso del metodo GET

In [ ]:
Url="https://services.bluekai.com/Services/WS/Ping?pid="+partner_id
signedUrl = bk.InputBuilder(bkuid=bkuid, bksecretkey=bksecretkey, url=Url, method="GET", data=None)
bk.doRequest(url=signedUrl,method="GET", data=None)

# Consulta de Audiencias
Para hacer esta consulta, podemos ver que la URL para pedir este servicio es

```http://services.bluekai.com/Services/WS/audiences?pid=```

en donde ```pid```hace referencia al partner id

In [ ]:
Url = "https://services.bluekai.com/Services/WS/audiences?pid="+partner_id
signedUrl = bk.InputBuilder(bkuid=bkuid, bksecretkey=bksecretkey, url=Url, method="GET", data=None)
Request = bk.doRequest(url=signedUrl, method="GET", data=None)
r1 = json.loads(Request)

In [ ]:
r1.keys()

In [ ]:
r1["total_count"]

In [ ]:
r1["audiences"]

In [ ]:
r1["audiences"][2]

In [ ]:
audiences = r1["audiences"]
Audience_names_ids = np.array([[i["name"],i["id"]] for i in audiences]) # get the name and the Id of avery audience
df = pd.DataFrame(data={"Id":Audience_names_ids[:,1],"Name Audience":Audience_names_ids[:,0]}) # set the dataframe to organise them

In [ ]:
df

# Consultando Audiencias una a una
Para esto usamos la misma API pero esta vez especificamos el Id de la Audiencia

In [ ]:
audienceID=df.Id[0]
Url = "https://services.bluekai.com/Services/WS/audiences/"+audienceID
signedUrl = bk.InputBuilder(bkuid, bksecretkey, Url, "GET", None)
Request = bk.doRequest(url=signedUrl, method="GET", data=None)
r1 = json.loads(Request)

In [ ]:
audienceID=df.Id[4]
Url = "https://services.bluekai.com/Services/WS/audiences/"+audienceID
signedUrl = bk.InputBuilder(bkuid, bksecretkey, Url, "GET", None)
Request = bk.doRequest(url=signedUrl, method="GET", data=None)
r1 = json.loads(Request)

In [ ]:
r1

# Publicando una audiencia
Para esto se hace uso de la misma API pero en esta ocación queremos es utilizar el metodo de POST, lo cual hace que esto sea un poco más complejo.

In [ ]:
values  = """{"partner.id":int,"name":str,"recency":90,"prospecting":true,"retargeting":true,"segments":{"AND":[{"AND":[{"OR":[{"cat":int,"freq":[ int , null]}]}]}]},"labels":["audience label"],"countryCodes":["CO"],"device_type":"all"}"""

Url = "https://services.bluekai.com/Services/WS/audiences?pid="+partner_id
nu = signatureInputBuilder(Url, "POST",values)
request = doRequest(nu,"POST", values)

# Realizando consulta de todas las Audiencias

In [ ]:
def Consult_Single_Audience(audienceID:str, pid:str, bkuid:str, bksecretkey:str):
    try:
        Url = "https://services.bluekai.com/Services/WS/audiences/"+audienceID
        signedUrl = bk.InputBuilder(bkuid, bksecretkey, Url, "GET", None)
        Request = bk.doRequest(url=signedUrl, method="GET", data=None)
        d = json.loads(Request)
        return [d["name"],d['reach'],d['price'],d['status']]
    except:
        return ["Error",0,0.0,""]

In [ ]:
Consult_Single_Audience(audienceID = df.Id[4],pid = partner_id, bkuid=bkuid, bksecretkey=bksecretkey)

In [ ]:
import multiprocessing
from functools import partial

In [ ]:
def GetData_Parallel(pid:str, bkuid:str, bksecretkey:str,List_ids_Audiences:list ,Processors:int = multiprocessing.cpu_count()):
    with multiprocessing.Pool(Processors) as p:
        Partial_function = partial(Consult_Single_Audience,pid=pid,bkuid=bkuid,bksecretkey= bksecretkey)
        request=np.array(p.map(Partial_function,List_ids_Audiences))
        df=pd.DataFrame(data=request,columns=["Name","Reach","Price","Status"])
    return df

In [ ]:
GetData_Parallel(List_ids_Audiences = df.Id,pid = partner_id, bkuid=bkuid, bksecretkey=bksecretkey)